# Introduction to pytket-braket

[Amazon Braket](https://aws.amazon.com/jp/braket/) provided by Amazon Web Services (AWS) allows developers to build, test, and run quantum algorithms on various quantum hardwares and simulators.  
`pytket-braket` is an extension to pytket that allows pytket circuits to be run on quantum devices and simulators on Amazon Braket.  

`pytket-braket` is available for Python 3.10, 3.11, 3.12 and 3.13, on Linux, MacOS and Windows. To install, run:
```
pip install pytket-braket
```

This will install `pytket` if it isn’t already installed, and add new classes and methods into the `pytket.extensions` namespace.

## Available Braket Backends via `pytket-braket`
`BraketBackend` can be used to interface with multiple simulators and quantum processors. `pytket-braket` supports the following quantum devices and simulators.
| DeviceName | DeviceType | Local/Cloud | Provider | region | 
| ---- | ---- | ---- | ---- | ---- |
| SV1 | SIMULATOR | Local | Amazon |  |
| DM1 | SIMULATOR | Local | Amazon |  |
| SV1 | SIMULATOR | Cloud | Amazon | eu-west-2, us-east-1, us-west-1, us-west-2 |
| TN1 | SIMULATOR | Cloud | Amazon | eu-west-2, us-east-1, us-west-2 |
| DM1 | SIMULATOR | Cloud | Amazon | eu-west-2, us-east-1, us-west-1, us-west-2 |
| Ankaa-3 | QPU | Cloud | Rigetti | us-west-1 |
| Garnet | QPU | Cloud | IQM | eu-north-1 |
| Aria-1 | QPU | Cloud | IonQ | us-east-1 |
| Aria-2 | QPU | Cloud | IonQ | us-east-1 |
| Forte-1 | QPU | Cloud | IonQ | us-east-1 |
| Forte-Enterprise-1 | QPU | Cloud | IonQ | us-east-1 |


### Braket Backend Parameters

 - **local** – use simulator running on local machine, default: False

 - **local_device** – name of local device (ignored if local=False) – e.g. “braket_sv” (default) or “braket_dm”.

 - **device** – device name from device ARN (e.g. “ionQdevice”, “Aspen-8”, …), default: “sv1”
 
 - **region** - e.g. 'eu-west-2' (for London), 'us-west-1' (for Northern California)

 - **s3_bucket** – name of S3 bucket to store results **Note:** this should always start with an "amazon-braket-" prefix

 - **s3_folder** – name of folder (“key”) in S3 bucket to store results in

 - **device_type** – device type from device ARN (e.g. “qpu”), default: “quantum-simulator”

 - **provider** – provider name from device ARN (e.g. “ionq”, “rigetti”, “oqc”, …), default: “amazon”

 - **aws_session** – braket AwsSession object, to pass credentials in if not configured on local machine

Lets use a simple GHZ circuit to demonstrate some features of the ``BraketBackend``. 

In [ ]:
from pytket import Circuit

def build_ghz_circ(n_qubits: int) -> Circuit:
    """
    Returns a circuit that prepares the n qubit GHZ state.
    """
    circ = Circuit(n_qubits)
    circ.H(0)
    for k in range(1, n_qubits):
        circ.CX(k - 1, k)
    circ.measure_all()
    return circ

$$ 
\begin{equation}
| \psi \rangle_{n} = \frac{1}{\sqrt{2}} \big( |0\rangle^{\otimes n} + |1\rangle^{\otimes n} \big)
\end{equation}
$$

In [ ]:
circ1 = build_ghz_circ(3) # define a 3 qubit GHZ circuit

In [ ]:
from pytket.circuit.display import render_circuit_jupyter
render_circuit_jupyter(circ1)

The pytket-braket extension offers the ability to convert circuits from a TKET format to a braket format and vice versa.

This is done using the ``tk_to_braket`` and ``braket_to_tk`` functions.

In [ ]:
from pytket.extensions.braket import tk_to_braket, braket_to_tk

In [ ]:
bk_circ = tk_to_braket(circ1) # convert from a TKET circuit to a braket circuit.
print(bk_circ[0])

In [ ]:
from pytket.extensions.braket import BraketBackend # import our BraketBackend

### Local simulator
If we set ``local=True`` then the only available simulator is the statevector simulator SV1. If used locally this can support a maximum of 26 qubits. If instead we use access this simulator remotely it can support up to 34 qubits.

In [ ]:
aws_backend_local = BraketBackend(local=True) #initialise local simulator

Lets use a simple GHZ circuit to demonstrate backends available in pytket-braket.

In [ ]:
aws_backend_local.backend_info # lets look at the device info and supported operations

Lets now compile our three qubit circuit to a form that can be executed on the local statevector simulator.

In [ ]:
compiled_circ = aws_backend_local.get_compiled_circuit(circ1) 
handle = aws_backend_local.process_circuit(compiled_circ)
result = aws_backend_local.get_result(handle)

In [ ]:
result # view our entire BackendResult Object - our statevector and density matrix will be calculated

In [ ]:
result.get_state() # our statevector - equal to the GHZ state up to a global phase

If we add some measurements to our circuit and an ``n_shots`` parameter to ``process_circuit`` then we can get shots based results instead. Our result is obtained by randomly sampling the statevector.

In [ ]:
handle2 = aws_backend_local.process_circuit(compiled_circ, n_shots=1000)
result2 = aws_backend_local.get_result(handle2)

In [ ]:
result2.get_counts()

We see that we have a roughly 50:50 distribution of shots in our two basis states as we would expect from measuring the GHZ state.

### Cloud simulator
Lets now take a look at how to use access remote cloud simulators with ``pytket-braket`` this requires passing additional parameters to the ``BraketBackend`` constructor. We have to specify a region as well as an s3 bucket and folder where our results will be stored.  
The bucket name must start with the characters ``amazon-braket-``. For inctance, ``amazon-braket-ABCDEFG``.

**Note:** The quantum devices that are available to use will depend on the region specified.  
Please see [documentation](https://eu-north-1.console.aws.amazon.com/braket/home?region=eu-north-1#/devices).

- Region for SV1  
eu-west-2, us-east-1, us-west-1, us-west-2
- Region for TN1  
eu-west-2, us-east-1, us-west-2
- Region for DM1  
eu-west-2, us-east-1, us-west-1, us-west-2

In [ ]:
my_s3_bucket = 'amazon-braket-XXXXXXXXXX'
my_region = 'us-west-2'

#### AWS credentials
You need to prepare your AWS credentials. See [AWS security credentials](https://docs.aws.amazon.com/IAM/latest/UserGuide/secrity-creds.html).  
We now read in your AWS credentials namely ``aws_access_key_id``, ``aws_secret_access_key``, ``s3_name``, ``bucket_key`` from the file ``aws-key`` in the folder.

In [ ]:
# Create a file named ``aws-key`` in the folder to store access keys and other information, then read it
path = 'aws-key'
f = open(path)
aws_access_key_id, aws_secret_access_key, s3_name, bucket_key= [s.strip() for s in f.readlines()]
f.close()

Create AwsSession using your AWS credentials.

In [ ]:
import boto3
from braket.aws.aws_session import AwsSession
my_boto_session = boto3.Session(aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key, region_name=my_region)
my_aws_session = AwsSession(boto_session=my_boto_session) # initialise an AwsSession

In [ ]:
# view the devices and simulators available in the us-west-1 region
my_aws_session.search_devices()

Lets initialise the tensor network simulator TN1. This is done by passing the ``device``, ``device_type`` and ``provider`` arguements to ``BraketBackend`` as they appear above.

This is a remote simulator that can simulate circuits of up to 50 qubits. This simulator uses tensor network contraction instead of the usual method of statevector simulation. This is especially effective for sparse quantum circuits. To read more about how TN1 works read the [documentation](https://docs.aws.amazon.com/braket/latest/developerguide/braket-devices.html#braket-simulator-tn1).

In [ ]:
aws_backend = BraketBackend(local=False,
                            device='tn1',
                            s3_bucket=s3_name,
                            s3_folder=bucket_key,
                            device_type='quantum-simulator',
                            provider='amazon',
                            aws_session=my_aws_session)

In [ ]:
from pytket.circuit.display import render_circuit_jupyter

ghz_circ = build_ghz_circ(3)
compiled_tn_circ = aws_backend.get_compiled_circuit(ghz_circ)
render_circuit_jupyter(compiled_tn_circ)

In [ ]:
handle = aws_backend.process_circuit(compiled_tn_circ, n_shots=100)
result = aws_backend.get_result(handle)

In [ ]:
result.get_counts()

### Rigetti device
Lets demonstrate accessing a real quantum processor with the ``BraketBackend``. We will use the ``Ankaa-3`` device from Rigetti. This is a superconducting device that can run circuits of up to 82 qubits.

**Note:** You need to set the option ``region_name`` of ``AwsSession`` for Rigetti device again.

In [ ]:
my_region = 'us-west-1' #Rigetti device region

In [ ]:
my_boto_session = boto3.Session(aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key, region_name=my_region)
my_aws_session = AwsSession(boto_session=my_boto_session) # initialise an AwsSession

Set backend as follows.  

In [ ]:
#initialise our BraketBackend for the Rigetti device
rigetti_backend = BraketBackend(local=False,
                                   region=my_region,
                                  device='Ankaa-3',
                                   s3_bucket=s3_name,
                                   s3_folder=bucket_key,
                                   device_type='qpu',
                                  provider='rigetti',
                                   aws_session=my_aws_session)

In [ ]:
# define a circuit to compile to the Ankaa-3 device
circ = build_ghz_circ(3)

In [ ]:
render_circuit_jupyter(circ)

In [ ]:
rigetti_backend.backend_info.gate_set

In [ ]:
rigetti_backend.required_predicates

In [ ]:
rigetti_compiled_circ = rigetti_backend.get_compiled_circuit(circ)
print("Is the circuit valid?", rigetti_backend.valid_circuit(rigetti_compiled_circ))
render_circuit_jupyter(rigetti_compiled_circ)

In [ ]:
rigetti_handle = rigetti_backend.process_circuit(rigetti_compiled_circ, n_shots=10)

In [ ]:
rigetti_result = rigetti_backend.get_result(rigetti_handle)
rigetti_result.get_counts()

### IQM device
When you use the IQM device, set backend as follows.  
**Note:** You need to set the option ``region_name`` of ``AwsSession`` for IQM device again.

In [ ]:
#my_region = 'us-west-1' #Rigetti device region
#my_region = 'us-east-1' #IonQ device region
my_region = 'eu-north-1' #IQM device region
my_boto_session = boto3.Session(aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key,
region_name=my_region)
my_aws_session = AwsSession(boto_session=my_boto_session)

In [ ]:
iqm_backend = BraketBackend(local=False,
                                  device='Garnet',
                                  region=my_region,
                                   s3_bucket=s3_name,
                                   s3_folder=bucket_key,
                                   device_type='qpu',
                                  provider='iqm',
                                   aws_session=my_aws_session)

In [ ]:
iqm_compiled_circ = iqm_backend.get_compiled_circuit(circ)
iqm_handle = iqm_backend.process_circuit(iqm_compiled_circ, n_shots=10)

In [ ]:
iqm_result = iqm_backend.get_result(iqm_handle)
iqm_result.get_counts()

### IonQ device
When you use the IonQ device, set backend as follows.
**Note:** You need to set the option ``region_name`` of ``AwsSession`` for IonQ device again.

In [ ]:
#my_region = 'us-west-1' #Rigetti device region
my_region = 'us-east-1' #IonQ device region
#my_region = 'eu-north-1' #IQM device region
my_boto_session = boto3.Session(aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key,region_name=my_region)
my_aws_session = AwsSession(boto_session=my_boto_session)

In [ ]:
ionq_backend = BraketBackend(local=False,
#                                  device='Aria-1',
#                                  device='Aria-2',
                                  device='Forte-1',
#                                  device='Forte-Enterprise-1',
                                  region=my_region,
                                   s3_bucket=s3_name,
                                   s3_folder=bucket_key,
                                   device_type='qpu',
                                  provider='ionq',
                                   aws_session=my_aws_session)

In [ ]:
ionq_compiled_circ = ionq_backend.get_compiled_circuit(circ)
ionq_handle = ionq_backend.process_circuit(ionq_compiled_circ, n_shots=10)

In [ ]:
ionq_result = ionq_backend.get_result(ionq_handle)
ionq_result.get_counts()